In [1]:
%%time

import os  
import shutil 
import random  
import re
from tqdm import tqdm
import threading
from extract_csi import get_scaled_csi, extract_csi
import scipy.signal as signal
import scipy.io as scio
import numpy as np
# 提取CSI中想要的部分
def processor(raw_file_path, f, save_path, max_file_len):
    file_name = os.path.join(raw_file_path, f)
    extracted_data = extract_csi(file_name)
    
    # print('processing {} the length of this file is:{}'.format(file_name, len(extracted_data)))
    try:
        tx, rx, sub = extracted_data[0]['csi'].shape
    except:
        print(file_name, len(extracted_data))
    data_csi = np.zeros((max_file_len, tx, rx, sub), dtype=np.complex64)
    # qinghua (1938, 1, 3, 30)  ;    self
    for i in range(len(extracted_data)):
        data_csi[i] = get_scaled_csi(extracted_data[i])
    data_csi = np.clip(np.abs(data_csi), 1e-8, 1e100).reshape(-1, tx, rx, 30)
    print(data_csi.shape)
    # data = np.zeros((data_csi.shape[0], tx, rx))  # N*4
#     for ant in range(tx, rx):  # 每个天线对上的CSI变化趋势相同,为节约计算这里选择天线对即可
#         data_csi_ant = data_csi[:, ant, :]
#         b, a = signal.butter(5, 4*2/30, 'low')
#         var_max = 0
#         s_max = None
#         for s in range(30):
#             carrier_data = signal.lfilter(b, a, data_csi_ant[:, s])  # N*1
#             length = len(carrier_data)
#             var_temp = np.var(carrier_data[length//5:3*length//5])
#             if var_max < var_temp:
#                 var_max = var_temp
#                 s_max = carrier_data
#         data[:, ant] = s_max
#     scio.savemat(os.path.join(save_path, f.replace('dat', 'mat')), {'csi': data})


raw_file_path, save_path = '/home/zut_csi/tomding/RCNN/data_gait/all/1', '/home/zut_csi/tomding/RCNN/data_processed/test'
#'/home/zut_csi/tomding/RCNN/da', '/home/zut_csi/tomding/RCNN/d'
X = {}
thread_list= []
max_file_len = 0
for f in tqdm(os.listdir(raw_file_path)):             
    if f.endswith('r1.dat'):
        file_len = len(extract_csi(os.path.join(raw_file_path, f)))
        if file_len > max_file_len:
            max_file_len = file_len
print(max_file_len)

100%|██████████| 3672/3672 [21:27<00:00,  2.85it/s]

3641
CPU times: user 21min 22s, sys: 8.5 s, total: 21min 30s
Wall time: 21min 27s


In [ ]:
%%time
%%time
# 提取CSI中想要的部分
def processor(raw_file_path, f, save_path, max_file_len):
    file_name = os.path.join(raw_file_path, f)
    extracted_data = extract_csi(file_name)
    
    # print('processing {} the length of this file is:{}'.format(file_name, len(extracted_data)))
    try:
        tx, rx, sub = extracted_data[0]['csi'].shape
    except:
        print(file_name, len(extracted_data))
    data_csi = np.zeros((max_file_len, tx, rx, sub), dtype=np.complex64)
    # qinghua (1938, 1, 3, 30)  ;    self
    for i in range(len(extracted_data)):
        data_csi[i] = get_scaled_csi(extracted_data[i])
    data_csi = np.clip(np.abs(data_csi), 1e-8, 1e100).transpose(1, 2, 3, 0)     
    for r in range(rx):  # 每个天线对上的CSI变化趋势相同,为节约计算这里选择天线对即可
        b, a = signal.butter(5, 4*2/30, 'low')
        var_max = 0
        for s in range(30):
            data_csi[0, r, s, :] = signal.lfilter(b, a, data_csi[0, r, s, :])  # N*1
    scio.savemat(os.path.join(save_path, f.replace('dat', 'mat')), {'csi': data_csi})


raw_file_path, save_path = '/home/zut_csi/tomding/RCNN/data_gait/all/1', '/home/zut_csi/tomding/RCNN/data_processed/test'
#'/home/zut_csi/tomding/RCNN/da', '/home/zut_csi/tomding/RCNN/d'
X = {}
thread_list= []
max_file_len = 3641
import multiprocessing
# 多进程任务
pool = multiprocessing.Pool(processes=8)
result= []
for f in tqdm(os.listdir(raw_file_path)):             
    if f.endswith('r1.dat'):
        result.append(pool.apply_async(processor, (raw_file_path, f, save_path, max_file_len, )))
pool.close()
print(1)
pool.join()
print(1)
ans = [res.get() for res in result]
print(ans)